In [9]:
from pytket.extensions.qiskit import (
    AerStateBackend,
    AerBackend,
    AerUnitaryBackend,
    IBMQBackend,
    IBMQEmulatorBackend,

)
from pytket.extensions.projectq import ProjectQBackend

In [4]:
from pytket import Circuit, Qubit
circ = Circuit(2)
circ.H(0)
circ.CX(0, 1)
print(circ)

<tket::Circuit, qubits=2, gates=2>


In [5]:
aer_state_b = AerStateBackend()
state_handle = aer_state_b.process_circuit(circ)
statevector = aer_state_b.get_result(state_handle).get_state()
print(statevector)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [6]:
aer_unitary_b = AerUnitaryBackend()
result = aer_unitary_b.run_circuit(circ)
print(result.get_unitary())

[[ 0.70710678+0.00000000e+00j  0.        +0.00000000e+00j
   0.70710678-8.65956056e-17j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.70710678+0.00000000e+00j
   0.        +0.00000000e+00j  0.70710678-8.65956056e-17j]
 [ 0.        +0.00000000e+00j  0.70710678+0.00000000e+00j
   0.        +0.00000000e+00j -0.70710678+8.65956056e-17j]
 [ 0.70710678+0.00000000e+00j  0.        +0.00000000e+00j
  -0.70710678+8.65956056e-17j  0.        +0.00000000e+00j]]


In [10]:
# test qasm file
# qasmfile = "/data/home/Ziming_Zhao/litt/quantum_code/hackathon/hackathon/ethz-hackathon22/benchmarking/circuits/000ab8f8-d668-11ea-8ff5-38f9d36dfbf2.qasm"

# H2 circuit
qasmfile = "../qasm_files/H2JordanWignerMapper.qasm"

from pytket.qasm import circuit_to_qasm, circuit_from_qasm
with open(qasmfile) as f:
    print(f.read())

c1 = circuit_from_qasm(qasmfile)
c = c1

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
x q[0];
sx q[1];
x q[2];
sx q[3];
h q[0];
h q[2];
cx q[1],q[0];
cx q[3],q[2];
rz(pi/2) q[0];
rz(pi/2) q[2];
cx q[1],q[0];
cx q[3],q[2];
h q[0];
sxdg q[1];
h q[2];
sxdg q[3];
sx q[0];
h q[1];
sx q[2];
h q[3];
cx q[1],q[0];
cx q[3],q[2];
rz(7*pi/2) q[0];
rz(7*pi/2) q[2];
cx q[1],q[0];
cx q[3],q[2];
sxdg q[0];
h q[1];
sxdg q[2];
h q[3];
sx q[0];
sx q[1];
h q[2];
sx q[3];
cx q[3],q[2];
cx q[2],q[1];
cx q[1],q[0];
rz(pi/8) q[0];
cx q[1],q[0];
sxdg q[0];
cx q[2],q[1];
h q[0];
sxdg q[1];
cx q[3],q[2];
h q[1];
h q[2];
sxdg q[3];
h q[2];
sx q[3];
cx q[3],q[2];
cx q[2],q[1];
cx q[1],q[0];
rz(pi/8) q[0];
cx q[1],q[0];
h q[0];
cx q[2],q[1];
h q[0];
h q[1];
cx q[3],q[2];
sx q[1];
h q[2];
sxdg q[3];
sx q[2];
sx q[3];
cx q[3],q[2];
cx q[2],q[1];
cx q[1],q[0];
rz(pi/8) q[0];
cx q[1],q[0];
h q[0];
cx q[2],q[1];
sx q[0];
sxdg q[1];
cx q[3],q[2];
h q[1];
sxdg q[2];
sxdg q[3];
sx q[2];
sx q[3];
cx q[3],q[2];
cx q[2],q[1];
cx q[1],q[0];
rz(12.173672) q[0];
cx 

In [11]:
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(c)

In [3]:
from pytket import Circuit, Qubit
from pytket.circuit.display import render_circuit_jupyter

circ = Circuit(2)
circ.H(0)
circ.CX(0, 1)
render_circuit_jupyter(circ)

In [ ]:
from pytket.passes import RepeatUntilSatisfiedPass
from pytket.passes import SequencePass, OptimisePhaseGadgets
from pytket.passes import DecomposeMultiQubitsCX

seqpass = SequencePass([DecomposeMultiQubitsCX(), OptimisePhaseGadgets()])

def no_CX(circ):
    return circ.n_gates_of_type(OpType.CX) == 0
circ = (
    Circuit(2)
    .CX(0, 1)
    .X(1)
    .CX(0, 1)
    .X(1)
    .CX(0, 1)
    .X(1)
    .CX(0, 1)
    .Z(1)
    .CX(1, 0)
    .Z(1)
    .CX(1, 0)
)
custom_pass = RepeatUntilSatisfiedPass(seqpass, no_CX)
cu = CompilationUnit(circ)
custom_pass.apply(cu)
circ1 = cu.circuit


In [7]:
from pytket.extensions.qiskit import AerBackend
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter

b = AerBackend()
b.default_compilation_pass

<bound method AerBackend.default_compilation_pass of <pytket.extensions.qiskit.backends.aer.AerBackend object at 0x7fd80b6b5b80>>

In [8]:
circ = Circuit(2).X(0).Y(1).CRz(0.5, 1, 0)
circ1 = b.get_compiled_circuit(circ)
render_circuit_jupyter(circ1)

In [9]:
b.required_predicates

[NoSymbolsPredicate,
 GateSetPredicate:{ CnX ZZPhase YYPhase XXPhase PhasedX Reset Rx H SXdg Measure Tdg SWAP T CCX Sdg S CU3 Y CU1 Unitary2qBox X SX noop RangePredicate Unitary1qBox Z Barrier U2 Ry Rz U3 U1 TK1 CX CY CZ CSWAP }]

In [10]:
b.valid_circuit(circ)

False

In [11]:
b.valid_circuit(circ1)

True

In [7]:
"""Circuit generation: tket example"""
from pytket.circuit import Circuit
c = Circuit(3, 2)
print(c.qubits)
print(c.bits)


[q[0], q[1], q[2]]
[c[0], c[1]]


In [8]:
from pytket.circuit import Qubit

# add units arbitary names
new_q1 = Qubit("alpha", 0)
new_q2 = Qubit("beta", 2, 1)
new_q3 = Qubit("gamma", (0, 0, 0))
c.add_qubit(new_q1)
c.add_qubit(new_q2)
c.add_qubit(new_q3)
print(c.qubits)

[alpha[0], beta[2, 1], gamma[0, 0, 0], q[0], q[1], q[2]]


In [9]:
c.add_q_register("delta", 4)
print(c.qubits)

[alpha[0], beta[2, 1], delta[0], delta[1], delta[2], delta[3], gamma[0, 0, 0], q[0], q[1], q[2]]


In [10]:
# add gate
c.CX(0, 1)

[CX q[0], q[1]; ]

In [11]:
print(c.qubits)

[alpha[0], beta[2, 1], delta[0], delta[1], delta[2], delta[3], gamma[0, 0, 0], q[0], q[1], q[2]]


In [12]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(c)

In [14]:
c.H(new_q1)
c.CX(Qubit("q", 1), new_q2)
c.Rz(0.5, new_q2)
print(c.get_commands())
render_circuit_jupyter(c)

[H alpha[0];, CX q[0], q[1];, H alpha[0];, CX q[1], beta[2, 1];, Rz(0.5) beta[2, 1];, CX q[1], beta[2, 1];, Rz(0.5) beta[2, 1];]


## Exporting to and importing from standard formats

In [15]:
c = Circuit(2)
c.CX(0, 1)
c1 = Circuit(2)
c1.CZ(1, 0)
c.append(c1)
print(c.get_commands())

[CX q[0], q[1];, CZ q[1], q[0];]


In [16]:
x, y = Qubit("x"), Qubit("y")
c = Circuit()
c.add_qubit(x)
c.add_qubit(y)
c.CX(x, y)
c1 = Circuit()
c1.add_qubit(x)
c1.add_qubit(y)
c1.CZ(y, x)
c.append(c1)
print(c.get_commands())

[CX x, y;, CZ y, x;]
